<a href="https://colab.research.google.com/github/xslittlemaggie/Other-ML-DL-Algorithm-notes/blob/master/Decision_Tree_from_scratch_to_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center> Realization of Decesion Tree from Scratch </center></h1>

## Step 1: Calculate gini impurity
The lower the better, more pure

In [0]:
from collections import Counter

def gini(labels):
    impurity = 1
    label_counts = Counter(labels)
    for label in label_counts:
      probability_of_label = label_counts[label]/len(labels)
      impurity -= probability_of_label ** 2
    return impurity

## Step 2: Calculate information gain

In [0]:
def information_gain(starting_labels, split_labels):
  info_gain = gini(starting_labels)
  for subset in split_labels:
    info_gain -= gini(subset) * len(subset)/len(starting_labels)
  return info_gain

## Step 3: Split data based on different features (column)

#### 3.1 split data based on each feature

In [0]:
cars = [['med', 'low', '3', '4', 'med', 'med'], 
        ['med', 'vhigh', '4', 'more', 'small', 'high'], 
        ['high', 'med', '3', '2', 'med', 'low'], 
        ['med', 'low', '4', '4', 'med', 'low'], 
        ['med', 'low', '5more', '2', 'big', 'med'], 
        ['med', 'med', '2', 'more', 'big', 'high'], 
        ['med', 'med', '2', 'more', 'med', 'med'], 
        ['vhigh', 'vhigh', '2', '2', 'med', 'low'], 
        ['high', 'med', '4', '2', 'big', 'low'], 
        ['low', 'low', '2', '4', 'big', 'med']]


car_labels = ['acc', 'acc', 'unacc', 'unacc', 'unacc', 'vgood', 'acc', 'unacc', 'unacc', 'good']

def split(dataset, labels, column):
    data_subsets = []
    label_subsets = []
    counts = list(set([data[column] for data in dataset]))
    counts.sort()
    for k in counts:
        new_data_subset = []
        new_label_subset = []
        for i in range(len(dataset)):
            if dataset[i][column] == k:
                new_data_subset.append(dataset[i])
                new_label_subset.append(labels[i])
        data_subsets.append(new_data_subset)
        label_subsets.append(new_label_subset)
    return information_gain(labels, label_subsets)


In [0]:
for i in range(6):
  data_subsets, label_subsets = split(cars, car_labels, i)
  print("The information gain for feature {} is: {:.4f}.".format(i + 1, information_gain(car_labels, label_subsets)))
 

The information gain for feature 1 is: 0.2733.
The information gain for feature 2 is: 0.0400.
The information gain for feature 3 is: 0.1067.
The information gain for feature 4 is: 0.3067.
The information gain for feature 5 is: 0.1500.
The information gain for feature 6 is: 0.2900.


#### 3.2 split data based on best feature

In [0]:
def find_best_split(dataset, labels):
    best_gain = 0
    best_feature = 0
    for feature in range(len(dataset[0])):
        data_subsets, label_subsets = split(dataset, labels, feature)
        gain = information_gain(labels, label_subsets)
        if gain > best_gain:
            best_gain, best_feature = gain, feature
    return best_feature, best_gain

In [0]:
find_best_split(cars, car_labels)

(3, 0.3066666666666667)

Thus, from the result above, when split the data with the fourth feature (indexed from 0), the information gain is highest. This is the best feature to split on.

## Step 4: Recursive Tree Building
Now that we can find the best feture to split the dataset, we can repeate this process again and again to create the full tree. This is cursive algorithm! We start with every data point from the trinign set, find the best feature to split the data, split the data based on that feature, and then recursively repeat the process again on each subset that was created from the split.

We'll stop the recursion when we can no longer find a feature that results in any information gain. In other words, we want to create a leaf of the tree when we can't find a way to split the data that makes purer subsets.

The leaf should keep track of the classes of the data points from the trining set that ended up in the leaf. In our inplementation, we'll use a Counter object to keep track of the counts of labels. 

In [0]:
def build_tree(data, labels):
  best_feature, best_gain = find_best_split(data, labels)
  if best_gain == 0:
    return Counter(labels)
  data_subsets, label_subsets = split(data, labels, best_feature)
  branches = []
  for i in range(len(data_subsets)):
    tree = build_tree(data_subsets[i], label_subsets[i])
    branches.append(tree)
  return branches
